In [20]:
import numpy as np

In [21]:
from nilearn._utils.niimg_conversions import _resolve_globbing
from nilearn.input_data import NiftiMasker
from nilearn.decomposition.base import mask_and_reduce
def prepare_data(func_filenames):
   
    imgs = _resolve_globbing(func_filenames)
    mask_img ='/home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/ADHD200_mask_152_4mm.nii.gz'
    masker = NiftiMasker(mask_img=mask_img, 
                         standardize=True,
                         detrend=1,
                         smoothing_fwhm=6.,
                         memory="/storage/nilearn_cache", 
                         memory_level=2)
    fmri_masked = masker.fit()

    data = mask_and_reduce(
        masker, imgs, confounds=None,
        reduction_ratio=1,
        random_state=0,
        memory_level=3,
        n_jobs=8)
    return data

In [22]:
import os
def prepare_filenames(dir_name):    
    func_filenames=[]
    for x in os.listdir(dir_name):
        file=dir_name+str(x)+'/sfnwmrda'+str(x)+'_session_1_rest_1.nii.gz'
        #print(file)
        if os.path.isfile(file):
            func_filenames.append(file)  
        else:
            print("missing "+file)  
    func_filenames=sorted(func_filenames)  

     # list of 4D nifti files for each subject  
    return func_filenames



In [23]:
import pandas as pd
def prepare_label(label_filenames):
    data = pd.read_csv(label_filenames)
    Y = data['DX'].values

    for index, item in enumerate(Y):
        if item>1:
            Y[index] = 1
    return Y


In [26]:
#sites=['KKI','NeuroIMAGE','OHSU','Pittsburgh','WashU','Peking_All']
sites=['Peking_All']
for site in sites:
    dir_name='/home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/'+site+ '/'
    Y=prepare_label(dir_name+site+'_phenotypic.csv')
    
    filenames = prepare_filenames(dir_name) 
    print("file numbers: ",len(filenames),"label numbers",len(Y))
    
    if len(filenames) != len(Y):
        raise Exception("X,Y not match")
    
    X=prepare_data(filenames)
    
    np.save(dir_name+'/All_Data',X)
    np.save(dir_name+'/All_Labels.npy',Y)

missing /home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/Peking_All/Peking_3_phenotypic.csv/sfnwmrdaPeking_3_phenotypic.csv_session_1_rest_1.nii.gz
missing /home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/Peking_All/Peking_2_phenotypic.csv/sfnwmrdaPeking_2_phenotypic.csv_session_1_rest_1.nii.gz
missing /home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/Peking_All/Peking_1_phenotypic.csv/sfnwmrdaPeking_1_phenotypic.csv_session_1_rest_1.nii.gz
missing /home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/Peking_All/motion/sfnwmrdamotion_session_1_rest_1.nii.gz
missing /home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/Peking_All/Peking_All_phenotypic.csv/sfnwmrdaPeking_All_phenotypic.csv_session_1_rest_1.nii.gz
file numbers:  194 label numbers 194
